<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Teste_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rotinas de Criação de Matriz de Embeddings


In [ ]:
! pip install keras==2.7.0
! pip show keras

     |████████████████████████████████| 1.3 MB 27.4 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 2.7.0 which is incompatible.


Name: keras
Version: 2.7.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: tensorflow, keras-vis


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy
import itertools
import nltk
import string, re
import pandas as pd
from numpy import array
from numpy import asarray
from numpy import loadtxt
from numpy import savetxt
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text, sequence

In [33]:
########## PATHS DE ARQUIVOS ##########
# 
PATH_CBOW_s300 = r"/content/drive/MyDrive/TCC/dados/word_embeddings/cbow_s300_modifyed.txt"

#
PATH_BASE_1_LEILA_CLASSIFICADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv" #entrada
PATH_MATRIZ_CBOW_300_BASE_1 = r"/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base1JuntaHateENotHateDaLeila_balanceada_CBOW300.CSV" #saida
#
PATH_BASE_2_LEILA_CLASSIFICADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv" #entrada
PATH_MATRIZ_CBOW_300_BASE_2_LEILA = r"/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_maior4_menor25_limpissima_balanceada_CBOW300.CSV" #saida

PATH_BASE_2_CLASSIFICADA = r"/content/drive/MyDrive/TCC/dados/processadas/Base2_classificada.csv" #entrada
PATH_MATRIZ_CBOW_300_BASE_2 = r"/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_classificada_CBOW300.csv" #saida

In [34]:
#DataFrame_Base= pd.read_csv(PATH_BASE_1_CLASSIFICADA)
DataFrame_Base = pd.read_csv(PATH_BASE_2_CLASSIFICADA)
DataFrame_Base.head()

,Unnamed: 0,text,label
0,1,retwet queria compartilhar antes gostaria sabe...,not_hate
1,2,gostei pulseira educação nossaaa muita elegânc...,not_hate
2,3,irão encontro beijing,not_hate
3,5,questionando é blogueiro blog juntos,not_hate
4,6,renan caloteiro desonesto vejam link abaixo,not_hate


In [35]:
palavras_unicas = list(DataFrame_Base['text'].str.split(' ', expand=True).stack().unique())
print(len(palavras_unicas))
# retira eventuais numeros. Numeros unicos por algum motivo matam o processo de filtragem. 
palavras_unicas = set(map(lambda x: re.findall(r'^\D+', x)[0] if re.findall(r'^\D+', x) != [] else "a" ,palavras_unicas))
print(len(palavras_unicas))

50996
50996


# Filtra os embeddings

In [36]:
# Código para filtrar o arquivo de embeddings.

embeddings_index = dict()

f = open(PATH_CBOW_s300, encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    if (word in palavras_unicas):
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()

In [37]:
def monta_linha(palavra, coeficientes):
    # Monta a linha com palavra _ coeficientes.
    
    linha = f'{palavra}'
    for coef in coeficientes:
        linha += f' {coef:.5f}'
    linha += '\n'
    
    return linha

In [38]:
lista_de_embeddings = []
for word in embeddings_index:
    coeficientes = embeddings_index[word]
    linha = monta_linha(word, coeficientes)
    lista_de_embeddings.append(linha)

In [39]:
embeddings_dict = {}
for i, line in enumerate(lista_de_embeddings):
    values = line.split()
    embeddings_dict[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [40]:
token = text.Tokenizer()
token.fit_on_texts(palavras_unicas)
#word_index: dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.
word_index = token.word_index

In [41]:
# create token-embedding mapping

lista_de_chaves = list(embeddings_dict.keys()) #palavras

tamanho_da_dimensao = len(embeddings_dict[lista_de_chaves[0]])

embedding_matrix = numpy.zeros((len(word_index) + 1, tamanho_da_dimensao ))

for word, i in word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [42]:
print(word_index)

{'namorado': 1, 'olko': 2, 'pombas': 3, 'jodanaj': 4, 'rito': 5, 'transformista': 6, 'maiacanetaémaispoderosaqueaespada': 7, 'argumentaê': 8, 'entropia': 9, 'boujour': 10, 'spears': 11, 'panteras': 12, 'silvane': 13, 'satisfatórios': 14, 'destacam': 15, 'opavaotemrazao': 16, 'camin': 17, 'percorrer': 18, 'escutas': 19, 'pecar': 20, 'tropeirão': 21, 'processa': 22, 'amanheci': 23, 'tapajós': 24, 'chers': 25, 'cassaachapatse': 26, 'suélem': 27, 'metades': 28, 'mastersofguitarbrazil': 29, 'choice': 30, 'contração': 31, 'engordar': 32, 'concessões': 33, 'necessitados': 34, 'oficialmente': 35, 'mulekes': 36, 'comete': 37, 'marchadodio': 38, 'padariagraoverde': 39, 'carimbadíssimo': 40, 'paulistano': 41, 'neworleans': 42, 'finando': 43, 'cagões': 44, 'confeitariasp': 45, 'demora': 46, 'retrasada': 47, 'divergência': 48, 'apresente': 49, 'aloysio': 50, 'visito': 51, 'comédia': 52, 'tra': 53, 'chopra': 54, 'irritou': 55, 'playstore': 56, 'recompor': 57, 'atenções': 58, 'gdó': 59, 'cobaia': 60,

In [43]:
print(type(embedding_matrix))
embedding_matrix.shape

<class 'numpy.ndarray'>


(50997, 300)

In [44]:
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.13437     0.07063     0.01832    ... -0.08156    -0.42243001
   0.01539   ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [-0.19501001  0.00599     0.45331001 ... -0.07847     0.07992
   0.21360999]
 [ 0.00875    -0.09458    -0.20144001 ... -0.01961    -0.25367999
   0.05939   ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [45]:
# save to csv file
savetxt(PATH_MATRIZ_CBOW_300_BASE_2, embedding_matrix, delimiter=',')

In [31]:
# data = loadtxt('data.csv', delimiter=',')
# # print the array
# print(data)